# Setting Up the Deployment/Test Environment


## Set Up Global Variables

First, change the azureSubscriptionId, resourceLocation, and resourceGroupName variables. The resourceGroupName defaults to a random unique name using the cell below.

### Note:
Please read about naming conventions and restrictions before continuing:  
- [Naming rules and restrictions for Azure resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/resource-name-rules)  
- [Create a unique name for a resource](https://docs.microsoft.com/en-us/azure/azure-resource-manager/templates/template-functions-string#uniquestring)  

In [1]:
# First, we will create a random name to append our Azure resource names. The name is not guaranteed to be unique, but doing this will increase the chances of generating a unique name. If the code snippet returns an error such as "resource name already exist", you should re-generate new name values.
import uuid 
nearlyUniqueName = uuid.uuid4().hex[:6].lower()

In [2]:
# Mandatory settings section
# ---------------------------

# Define the ID of the Azure subscription that will be used
# You can find your subscription ID at https://account.azure.com/subscriptions
azureSubscriptionId = "e78c95ce-3ed7-49eb-a19e-8ebcc73bb608"

# Define the datacenter location of the Azure services that will be created. 
# To get the list of possible resource locations, use the following commands on an Azure CLI-enabled terminal window:
# az account list-locations
# Please select a region to deploy resources from this list: canadaeast, centralus, eastus2, francecentral, japanwest, northcentralus, switzerlandnorth, uksouth, westcentralus, westus2, eastus2euap, centraluseuap.
resourceLocation = "southcentralus" 

# Define the name of the resource group you will use to contain all of the Azure services that will be created.
# Please use a globally unique resource group name
resourceGroupName = "lvasample" + nearlyUniqueName
# resourceGroupName = "<RESOURCE_GROUP_NAME>"  # Uncomment this command if you want to create your own Resource Group Name

# Optional settings section
# ---------------------------

# Azure Container Registry service to host your Docker container images
acrServiceName = resourceGroupName+'contreg'

# Azure IoT Hub service
iotHubServiceName = resourceGroupName+'iothub'

# Name of the container image for ML solution
containerImageName = resourceGroupName+'aimodule'

# Define a unique IoT Edge device ID
iotDeviceId = resourceGroupName+'iotdevid'

# Azure Media services
mediaServiceName = resourceGroupName+'ams'

# Azure Storage services
storageServiceName = resourceGroupName+'storage'

# Define a folder name that will be the host to all your ML solution files
isSolutionPath = "inferenceserver"

## Check Azure CLI Version
Azure CLI is required to run this sample. Please check if Azure CLI has been installed on your machine. If it has not, then visit the [requirements for running LVA on Jupyter](01_requirements.md) page. 

In [3]:
!az --version

azure-cli                         2.0.80 *

command-modules-nspkg              2.0.3
core                              2.0.80 *
nspkg                              3.0.4
telemetry                          1.0.4

Extensions:
azure-cli-ml                      1.0.85

Python location '/opt/az/bin/python3'
Extensions directory '/opt/az/extensions'

Python (Linux) 3.6.5 (default, Jan 13 2020, 05:04:39) 
[GCC 7.4.0]

Legal docs and information: aka.ms/AzureCliLegal



You have 2 updates available. Consider updating your CLI installation. Instructions can be found at https://docs.microsoft.com/en-us/cli/azure/install-azure-cli

Please let us know how we are doing: https://aka.ms/clihats


## Login to Azure
> <span style="color:red; font-weight:bold"> [!IMPORTANT] </span>  
> Run the following command and **follow the instructions** in the corresponding output cell. You should enter the authentication code to allow this notebook kernel to access Azure resources. You can run the `az logout` command or stop the kernel to logout.

In [4]:
%%bash --out output
az account show &> /dev/null
if [ "$?" != "0" ]; then echo 'login'; fi

In [5]:
if (output != ''):
    print('You must sign in to Azure. Please follow the instructions below.')
    !az login -o None
    output = ''
else:
    print('There is an active Azure login session.')

There is an active Azure login session.


> <span> [!NOTE] </span>  
> If the code cells above return errors, you may be using a terminal that does not support Bash commands. If you are using a machine running Windows, you can try either of the following solutions:
> <br><br>Option 1: Turn on the [Windows Subsystem for Linux](https://code.visualstudio.com/remote-tutorials/wsl/enable-wsl) setting. Then, download [Ubuntu 18.04](https://docs.microsoft.com/en-us/windows/wsl/install-win10#install-your-linux-distribution-of-choice) and [switch the terminal setting](https://code.visualstudio.com/docs/remote/wsl#_open-a-remote-folder-or-workspace) in VSCode so that the WSL terminal is used.
> <br><br>Option 2: Install [Git Bash](https://git-scm.com/downloads) and [switch the terminal setting](https://code.visualstudio.com/docs/editor/integrated-terminal) in VSCode so that the Bash terminal is used.

## Set Active Azure Subscription
In case you have more than one Azure subscription, we will set the one you specified in the **azureSubscriptionId** variable as the default to be used.

In [6]:
!az account set --subscription $azureSubscriptionId

## Install the dotenv Python Library 

We will now be installing the [dotenv Python library](https://pypi.org/project/python-dotenv/). This library reads the key-value pair from a .env file and adds them to the environment variables. This library is great for managing app settings during development and sharing global variables among code files.

> <span style="color:red; font-weight: bold;"> [!IMPORTANT] </span>  
> Depending on your version of Python, you may need to switch between `pip` and `pip3`. Check the output of the cell below for possible errors and make the appropriate switch if needed.

In [7]:
%%bash
if python3 -c 'import pkgutil; exit(not pkgutil.find_loader("dotenv"))';
then
	echo "dotenv package found. Skipping re-installation...";
else
	echo "dotenv package not found. Installing...";
    pip install python-dotenv;
fi

dotenv package found. Skipping re-installation...


## Save Global Variables in .env File
Check if a .env file already exist. If not, create a new one. The .env file is where we will store our variables to share with other code books of this sample.

In [8]:
from pathlib import Path
from dotenv import set_key, get_key, find_dotenv

envPath = find_dotenv()
if envPath=="":
    Path(".env").touch()
    envPath = find_dotenv()
    
print(".env file path: {0}".format(envPath))

.env file path: /home/visionadmin/source/live-video-analytics/utilities/video-analysis/notebooks/common/.env


In [9]:
set_key(envPath, "NEARLY_UNIQUE_NAME", nearlyUniqueName)
set_key(envPath, "SUBSCRIPTION_ID", azureSubscriptionId)
set_key(envPath, "RESOURCE_LOCATION", resourceLocation)
set_key(envPath, "RESOURCE_GROUP", resourceGroupName)
set_key(envPath, "ACR_SERVICE_NAME", acrServiceName)
set_key(envPath, "CONTAINER_IMAGE_NAME", containerImageName)
set_key(envPath, "IOT_HUB_SERVICE_NAME", iotHubServiceName)
set_key(envPath, "IOT_DEVICE_ID", iotDeviceId)
set_key(envPath, "AMS_ACCOUNT", mediaServiceName)
set_key(envPath, "STORAGE_SERVICE_NAME", storageServiceName)
set_key(envPath, "IS_SOLUTION_PATH", isSolutionPath)

(True, 'IS_SOLUTION_PATH', 'inferenceserver')

## Next Steps
If all the code cells above have successfully finished running, check the [.env file](.env) to see if all the variables set above have been included. Then, return to the Readme page to continue.   